In [1]:
import pyspark

In [7]:
from pyspark.sql import SparkSession

In [2]:
sc = pyspark.SparkContext("local[*]")

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/02/04 19:56:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [9]:
spark = SparkSession.builder\
      .master("local[1]")\
      .appName("exam_solutions")\
      .getOrCreate() 

In [11]:
rdd = spark.sparkContext.textFile("data.txt")

In [12]:
rdd.first()

'Name Surname'

In [32]:
# TransactionID:Int | Timestamp:Datetime | UserID:String | CompanySymbol:String | Volume:Int| Action:String
transactions = [('1991-04-01',0,'Smith',20,'Buy'),
  ('2000-05-19',1,'Ford',100,'Sell'),
  ('1978-09-05',2,'Williams',12,'Buy'),
  ('1967-12-01',3,'Jones',27,'Buy'),
  ('1980-02-17',4,'Brown',160,'Sell')
]

transactions_columns = ["Timestamp","UserID","CompanySymbol","Volume","Action"]
df_trans = spark.createDataFrame(data=transactions, schema = transactions_columns)

In [33]:
df_trans.show()

+----------+------+-------------+------+------+
| Timestamp|UserID|CompanySymbol|Volume|Action|
+----------+------+-------------+------+------+
|1991-04-01|     0|        Smith|    20|   Buy|
|2000-05-19|     1|         Ford|   100|  Sell|
|1978-09-05|     2|     Williams|    12|   Buy|
|1967-12-01|     3|        Jones|    27|   Buy|
|1980-02-17|     4|        Brown|   160|  Sell|
+----------+------+-------------+------+------+



In [37]:
# CompanySymbol:String | Timestamp:Datetime | ValuePerUnit_EURO: Float
prices = [('Smith', '2022-02-01', 12.12),
         ('Ford', '2022-02-02', 22.22),
         ('Williams', '2022-02-01', 43.43),
         ('Jones', '2022-02-04', 35.35),
         ('Brown', '2022-02-03', 56.56)
]

prices_columns = ["CompanySymbol", "Timestamp", "ValuePerUnit_EURO"]
df_prices = spark.createDataFrame(data = prices, schema = prices_columns)

In [38]:
df_prices.show()

+-------------+----------+-----------------+
|CompanySymbol| Timestamp|ValuePerUnit_EURO|
+-------------+----------+-----------------+
|        Smith|2022-02-01|            12.12|
|         Ford|2022-02-02|            22.22|
|     Williams|2022-02-01|            43.43|
|        Jones|2022-02-04|            35.35|
|        Brown|2022-02-03|            56.56|
+-------------+----------+-----------------+

